In [2]:
!kaggle datasets download -d sunilthite/cat-or-dog-image-classification

Dataset URL: https://www.kaggle.com/datasets/sunilthite/cat-or-dog-image-classification
License(s): other
 98% 587M/599M [00:09<00:00, 88.2MB/s]
100% 599M/599M [00:09<00:00, 68.7MB/s]


In [3]:
import os

# Create a directory to store the dataset if it doesn't exist
os.makedirs('cat_or_dog_dataset', exist_ok=True)

# Extract the dataset
!unzip cat-or-dog-image-classification.zip -d cat_or_dog_dataset


Streaming output truncated to the last 5000 lines.
  inflating: cat_or_dog_dataset/Train/Dog/dog.5146.jpg  
  inflating: cat_or_dog_dataset/Train/Dog/dog.5147.jpg  
  inflating: cat_or_dog_dataset/Train/Dog/dog.5148.jpg  
  inflating: cat_or_dog_dataset/Train/Dog/dog.5149.jpg  
  inflating: cat_or_dog_dataset/Train/Dog/dog.515.jpg  
  inflating: cat_or_dog_dataset/Train/Dog/dog.5150.jpg  
  inflating: cat_or_dog_dataset/Train/Dog/dog.5151.jpg  
  inflating: cat_or_dog_dataset/Train/Dog/dog.5153.jpg  
  inflating: cat_or_dog_dataset/Train/Dog/dog.5154.jpg  
  inflating: cat_or_dog_dataset/Train/Dog/dog.5155.jpg  
  inflating: cat_or_dog_dataset/Train/Dog/dog.5156.jpg  
  inflating: cat_or_dog_dataset/Train/Dog/dog.5157.jpg  
  inflating: cat_or_dog_dataset/Train/Dog/dog.5158.jpg  
  inflating: cat_or_dog_dataset/Train/Dog/dog.5159.jpg  
  inflating: cat_or_dog_dataset/Train/Dog/dog.516.jpg  
  inflating: cat_or_dog_dataset/Train/Dog/dog.5160.jpg  
  inflating: cat_or_dog_dataset/Train/D

In [15]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report,mean_squared_error
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [5]:
# Create an ImageDataGenerator with scaling and data augmentation
datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Scale pixel values to the range [0, 1]
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

# Define batch size and image size
batch_size = 256  # Adjust the batch size according to your needs
image_size = (64, 64)  # Resize to 64x64 to reduce computational load


In [6]:
# Load and preprocess the images using the generator
train_data_generator = datagen.flow_from_directory(
    "/content/cat_or_dog_dataset/Train",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=True
)

test_data_generator = datagen.flow_from_directory(
    "/content/cat_or_dog_dataset/Test",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=True
)


Found 23650 images belonging to 2 classes.
Found 3863 images belonging to 2 classes.


In [7]:
def extract_features(data_generator):
    features = []
    labels = []
    for i in range(len(data_generator)):
        imgs, lbls = data_generator.next()
        for img, lbl in zip(imgs, lbls):
            features.append(img.flatten())
            labels.append(lbl)
    return np.array(features), np.array(labels)

X_train, y_train = extract_features(train_data_generator)
X_test, y_test = extract_features(test_data_generator)


In [9]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Standardize features
    ('sgd', SGDClassifier(loss='hinge'))  # SVM with SGD
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('sgd', SGDClassifier())])

In [11]:
# Step 4: Evaluate the model
y_pred = pipeline.predict(X_test)


In [12]:
# Calculate Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')


Accuracy: 0.539477090344292


In [13]:
# Generate Classification Report
print('Classification Report:')
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

         0.0       0.54      0.55      0.55      1944
         1.0       0.54      0.53      0.53      1919

    accuracy                           0.54      3863
   macro avg       0.54      0.54      0.54      3863
weighted avg       0.54      0.54      0.54      3863



In [16]:
# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 0.46052291989326477


In [17]:
print(train_data_generator.class_indices)
class_labels = ["Cat", "Dog"]

{'Cat': 0, 'Dog': 1}
